# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from IPython.display import display
def seperate_team_from(cities, sport):
    def seperate(s):
        ms=[]
        fromm=0
        for i in range(len(s)-1):
            if(s[i].islower() and s[i+1].isupper()):
                ms.append(s[fromm:i+1])
                fromm=i+1
        ms.append(s[fromm:])
        return ms

    def get_team(row):
        nonlocal sport
        row['team']=seperate(row[sport])
        return row
    
    cities=cities.apply(get_team, axis=1).explode('team')
    return cities.drop(['NFL', 'MLB', 'NBA', 'NHL'], axis=1)


def get_grouped_population_ratio(cities):
    cities['Population (2016 est.)[8]']=cities['Population (2016 est.)[8]'].astype('int64')
    cities_grobj=cities.groupby('Metropolitan area')
    #lambda rows: rows[0]
    population=list(cities_grobj['Population (2016 est.)[8]'].agg(lambda rows: rows[0]))#np.nanmean
    ratio=list(cities_grobj['WL ratio'].agg(np.nanmean))
    return population, ratio


def get_WL_ratio_from_2018(df):
    df=df[['team', 'W', 'L', 'year']]
    df=df[df['year']>=2018]
    df=df.astype({'W': 'int64', 'L': 'int64'})
    df['WL ratio']=df['W']/(df['W']+df['L'])
    return df[['team', 'WL ratio']]


cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities=cities.replace('[ ]*\[.*\]', '', regex=True)
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders,GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears,CubsWhite Sox,Bulls,Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [2]:
def add_team_last_words(cities):
    def check_conflict(cities):
        unique=cities['Team (last word)'].unique()
        if(len(unique)<cities.shape[0]):
            appearence=dict(zip(list(cities['Team (last word)'].unique()), [0]*len(unique)))
            for team in cities['Team (last word)']:
                appearence[team]+=1
            samewords=[word for word in appearence.keys() if appearence[word]>1]
            cities=cities.reset_index(drop=True)
            for word in samewords:
                mask=(cities['Team (last word)']==word)
                last2words_ser=cities[mask]['team'].str.extract('([^ ]* *[^ ]*)$')[0]
                for index in list(last2words_ser.index):
                    last2words=last2words_ser[index]
                    cities.loc[index, 'Team (last word)']=last2words
        return cities
    
    cities['Team (last word)']=cities['team'].str.extract('([^ ]*)$')
    return check_conflict(cities)

,Team (last word),team
0,york,new york
1,white sox,white sox
2,lycapower,lycapower
3,black sox,black sox


In [3]:
cities1=seperate_team_from(cities, 'NHL')
cities1=add_team_last_words(cities1)
cities1.set_index('Team (last word)', inplace=True)

cities1.head()

,Metropolitan area,Population (2016 est.)[8],team
Team (last word),,,
Rangers,New York City,20153634,Rangers
Islanders,New York City,20153634,Islanders
Devils,New York City,20153634,Devils
Kings,Los Angeles,13310447,Kings
Ducks,Los Angeles,13310447,Ducks


In [4]:
nhl_df=pd.read_csv("assets/nhl.csv")

nhl_df=nhl_df[nhl_df['team'].str.contains('Division')==False]
nhl_df=get_WL_ratio_from_2018(nhl_df)
nhl_df=nhl_df.replace('\*$', '', regex=True)

nhl_df=add_team_last_words(nhl_df)
nhl_df.set_index('Team (last word)', inplace=True)
nhl_df

,team,WL ratio
Team (last word),,
Lightning,Tampa Bay Lightning,0.701299
Bruins,Boston Bruins,0.714286
Leafs,Toronto Maple Leafs,0.653333
Panthers,Florida Panthers,0.594595
Wings,Detroit Red Wings,0.434783
Canadiens,Montreal Canadiens,0.420290
Senators,Ottawa Senators,0.394366
Sabres,Buffalo Sabres,0.357143
Capitals,Washington Capitals,0.653333


df=pd.merge(cities1, nhl_df, how='left', left_index=True, right_index=True)
print(df[df['WL ratio'].isna()].index)
df=pd.merge(cities1, nhl_df, how='right', left_index=True, right_index=True)
print(df[df['Metropolitan area'].isna()].index)

In [5]:
cities_NHL=pd.merge(cities1, nhl_df, how='left', left_index=True, right_index=True).dropna()
cities_NHL.head()

,Metropolitan area,Population (2016 est.)[8],team_x,team_y,WL ratio
Team (last word),,,,,
Avalanche,Denver,2853077,Avalanche,Colorado Avalanche,0.589041
Blackhawks,Chicago,9512999,Blackhawks,Chicago Blackhawks,0.458333
Blues,St. Louis,2807002,Blues,St. Louis Blues,0.578947
Bruins,Boston,4794447,Bruins,Boston Bruins,0.714286
Canadiens,Montreal,4098927,Canadiens,Montreal Canadiens,0.420290


In [6]:
def nhl_correlation(): 
    # YOUR CODE HERE
    population, ratio=get_grouped_population_ratio(cities_NHL)
    population_by_region = population
    win_loss_by_region = ratio # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nhl_correlation()

0.012486162921209923

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [34]:
cities2=seperate_team_from(cities, 'NBA')
cities2=add_team_last_words(cities2)
cities2.set_index('Team (last word)', inplace=True)

nba_df=pd.read_csv("assets/nba.csv")
nba_df=nba_df[nba_df['team'].str.contains('Division')==False]
nba_df=get_WL_ratio_from_2018(nba_df)
nba_df['team']=nba_df['team'].replace('[^a-zA-Z]*$', '', regex=True)
nba_df=add_team_last_words(nba_df)
nba_df.set_index('Team (last word)', inplace=True)

cities_NBA=pd.merge(cities2, nba_df, how='left', left_index=True, right_index=True).dropna()

def nba_correlation():
    # YOUR CODE HERE
    population, ratio=get_grouped_population_ratio(cities_NBA)
    
    population_by_region = population # pass in metropolitan area population from cities
    win_loss_by_region = ratio # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [35]:
cities3=seperate_team_from(cities, 'MLB')
cities3=add_team_last_words(cities3)
cities3.set_index('Team (last word)', inplace=True)
mlb_df=pd.read_csv("assets/mlb.csv")
mlb_df=get_WL_ratio_from_2018(mlb_df)
mlb_df=add_team_last_words(mlb_df)
mlb_df.set_index('Team (last word)', inplace=True)

cities_MLB=pd.merge(cities3, mlb_df, how='left', left_index=True, right_index=True).dropna()
def mlb_correlation(): 
    # YOUR CODE HERE
    population, ratio=get_grouped_population_ratio(cities_MLB)
    
    population_by_region = population # pass in metropolitan area population from cities
    win_loss_by_region = ratio # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [55]:
cities4=seperate_team_from(cities, 'NFL')
cities4=add_team_last_words(cities4)
cities4.set_index('Team (last word)', inplace=True)

nfl_df=pd.read_csv("assets/nfl.csv")
nfl_df=nfl_df[nfl_df['team'].str.contains('AFC|NFC')==False]
nfl_df=get_WL_ratio_from_2018(nfl_df)
nfl_df=nfl_df.replace('[^a-zA-Z]*$', '', regex=True)
nfl_df=add_team_last_words(nfl_df)
nfl_df.set_index('Team (last word)', inplace=True)
display(cities4)
display(nfl_df)
cities_NFL=pd.merge(cities4, nfl_df, how='left', left_index=True, right_index=True).dropna()
def nfl_correlation(): 
    # YOUR CODE HERE
    population, ratio=get_grouped_population_ratio(cities_NFL)
    
    population_by_region = population # pass in metropolitan area population from cities
    win_loss_by_region = ratio # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]


    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
nfl_correlation()

,Metropolitan area,Population (2016 est.)[8],team
Team (last word),,,
Giants,New York City,20153634,Giants
Jets,New York City,20153634,Jets
Rams,Los Angeles,13310447,Rams
Chargers,Los Angeles,13310447,Chargers
49ers,San Francisco Bay Area,6657982,49ers
Raiders,San Francisco Bay Area,6657982,Raiders
Bears,Chicago,9512999,Bears
Cowboys,Dallas–Fort Worth,7233323,Cowboys
Redskins,"Washington, D.C.",6131977,Redskins


,team,WL ratio
Team (last word),,
Patriots,New England Patriots,0.687500
Dolphins,Miami Dolphins,0.437500
Bills,Buffalo Bills,0.375000
Jets,New York Jets,0.250000
Ravens,Baltimore Ravens,0.625000
Steelers,Pittsburgh Steelers,0.600000
Browns,Cleveland Browns,0.466667
Bengals,Cincinnati Bengals,0.375000
Texans,Houston Texans,0.687500


0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [1]:
#not done yet

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values